Colonnes disponibles : ['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2', 'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct', 'co2_including_luc', 'co2_including_luc_growth_abs', 'co2_including_luc_growth_prct', 'co2_including_luc_per_capita', 'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy', 'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2', 'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita', 'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2', 'cumulative_co2_including_luc', 'cumulative_coal_co2', 'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2', 'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita', 'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2', 'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita', 'land_use_change_co2', 'land_use_change_co2_per_capita', 'methane', 'methane_per_capita', 'nitrous_oxide', 'nitrous_oxide_per_capita', 'oil_co2', 'oil_co2_per_capita', 'other_co2_per_capita', 'other_industry_co2', 'primary_energy_consumption', 'share_global_cement_co2', 'share_global_co2', 'share_global_co2_including_luc', 'share_global_coal_co2', 'share_global_cumulative_cement_co2', 'share_global_cumulative_co2', 'share_global_cumulative_co2_including_luc', 'share_global_cumulative_coal_co2', 'share_global_cumulative_flaring_co2', 'share_global_cumulative_gas_co2', 'share_global_cumulative_luc_co2', 'share_global_cumulative_oil_co2', 'share_global_cumulative_other_co2', 'share_global_flaring_co2', 'share_global_gas_co2', 'share_global_luc_co2', 'share_global_oil_co2', 'share_global_other_co2', 'share_of_temperature_change_from_ghg', 'temperature_change_from_ch4', 'temperature_change_from_co2', 'temperature_change_from_ghg', 'temperature_change_from_n2o', 'total_ghg', 'total_ghg_excluding_lucf', 'trade_co2', 'trade_co2_share']


In [32]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro

In [33]:
# === 📂 2. Chargement des données ===
df = pd.read_csv("/Users/yoavcohen/Desktop/Memoire_DU_SDA/Mémoire/Data/owid-co2-data.csv")
print("Colonnes disponibles :", df.columns.tolist())


Colonnes disponibles : ['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2', 'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct', 'co2_including_luc', 'co2_including_luc_growth_abs', 'co2_including_luc_growth_prct', 'co2_including_luc_per_capita', 'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy', 'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2', 'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita', 'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2', 'cumulative_co2_including_luc', 'cumulative_coal_co2', 'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2', 'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita', 'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2', 'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita', 'land_use_change_co2', 'land_use_change_co2_per_capita', 'methane', 'methane_per_capita', 'nitrous_oxide', 'nitr

In [44]:
# Variables à garder (excluant population, gdp brut, year, country, iso_code)
selected_vars = [
    'co2_per_capita', 'energy_per_capita', 'cement_co2_per_capita', 'gas_co2_per_capita',
    'oil_co2_per_capita', 'other_co2_per_capita', 'methane_per_capita', 'nitrous_oxide_per_capita',
    'ghg_per_capita', 'land_use_change_co2_per_capita', 'co2_per_gdp', 'energy_per_gdp',
    'co2_including_luc_per_gdp', 'total_ghg', 'cement_co2', 'coal_co2', 'gas_co2', 'oil_co2',
    'other_industry_co2', 'primary_energy_consumption', 'temperature_change_from_ghg', 'trade_co2',
     'energy_per_gdp', 'co2_growth_abs', 'share_global_cumulative_co2','trade_co2_share'
]

# Nettoyage : on garde ces colonnes + 'gdp' + 'population' (pour calcul PIB/hab)
df_world_sel = df[selected_vars + ['gdp', 'population']].dropna()

# Calcul PIB par habitant target
df_world_sel['gdp_per_capita'] = df_world_sel['gdp'] / df_world_sel['population']

# On enlève colonnes non nécessaires
df_world_sel.drop(columns=['gdp', 'population'], inplace=True)

In [45]:
# Modèle 3 : gdp_per_capita ~ population + primary_energy_consumption + methane_per_capita + temperature_change_from_ghg
X3 = df_world_sel[["primary_energy_consumption", "methane_per_capita", "gdp_per_capita", 'energy_per_gdp', 'trade_co2']]
X3 = sm.add_constant(X3)
y3 = df_world_sel["co2_per_capita"]

model3 = sm.OLS(y3, X3).fit()
print("\n=== Modèle 3 : GDP par habitant ~ population + énergie primaire + méthane per capita + changement température GES ===")
print(model3.summary())




=== Modèle 3 : GDP par habitant ~ population + énergie primaire + méthane per capita + changement température GES ===
                            OLS Regression Results                            
Dep. Variable:         co2_per_capita   R-squared:                       0.415
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     200.4
Date:                Wed, 06 Aug 2025   Prob (F-statistic):          1.64e-161
Time:                        20:44:54   Log-Likelihood:                -3716.5
No. Observations:                1416   AIC:                             7445.
Df Residuals:                    1410   BIC:                             7477.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.9

In [46]:
# Modèle 3 : gdp_per_capita ~ population + primary_energy_consumption + methane_per_capita + temperature_change_from_ghg
X3 = df_world_sel[["gdp_per_capita", 'trade_co2_share', 'share_global_cumulative_co2']]
X3 = sm.add_constant(X3)
y3 = df_world_sel["co2_growth_abs"]

model3 = sm.OLS(y3, X3).fit()
print("\n=== Modèle 3 : GDP par habitant ~ population + énergie primaire + méthane per capita + changement température GES ===")
print(model3.summary())



=== Modèle 3 : GDP par habitant ~ population + énergie primaire + méthane per capita + changement température GES ===
                            OLS Regression Results                            
Dep. Variable:         co2_growth_abs   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     59.99
Date:                Wed, 06 Aug 2025   Prob (F-statistic):           1.67e-36
Time:                        20:44:57   Log-Likelihood:                -8636.3
No. Observations:                1416   AIC:                         1.728e+04
Df Residuals:                    1412   BIC:                         1.730e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.